# Group Equivariant Graph Tutorial

## Introduction

In this tutorial, we present the different functionnalities of our implementation of the group equivariant graph. Such a graph is a k-NN graph representing the structure of a group, a set equipped with a binary operation called the group product. Each vertex of the graph represents an element of the group and the edge's weights corresponds to the Riemannian distance between these element. 

In [ ]:
import torch
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
from gechebnet.graphs.viz import visualize_graph_neighborhood, visualize_graph

## Package

The `gechebnet` package implements such a graph and have the basic functions to interact with them. For the most part, it uses PyTorch for tensor operations and PyKeops to achieve a memory and time efficient graph construction.

## Graph class

The main class of the package is the `Graph` class. It stores a graph in a very efficient way using `node_index`, `edge_index` and `edge_weight` attributes. Very easily, we can get important properties of a graph with `num_nodes` or `num_edges`. Other method, properties and attributes are available, we will see them later. However, this class just gives the main structure of a graph, the group equivariant graph inherits from this class.

An important remark is that our graphs are necessarily symmetric, that's why we refer the `knn` parameter as the **maximum** number of neighbours of a vertex.

In [ ]:
from gechebnet.graphs.graphs import SE2GEGraph, SO3GEGraph

## SE(2) Group Equivariant Graph

The SE(2) group equivariant graph is the first group equivariant graph we have implemented. To create such a graph, different variables have to be speficied:
- `nx`: the discretization on the x axis, i.e. the number of points to uniformly sample in the x direction.
- `ny`: the discretization on the y axis, i.e. the number of points to uniformly sample in the y direction.
- `nsym`: the discretization on the symmetry axis, i.e. the number of points to uniformly sample in the orientation domain.
- `knn`: the maximum number of neighbors of each vertex. 
- `sigmas`: the anisotropy's parameters for the computation of the Riemannian distance between vertices.
- `weight_kernel`: the weight kernel, that is a function taking as input the squared Riemannian distance and returning a weight.
- `kappa`: the edge's compression rate, the rate of edges to drop during random compression.

In [ ]:
from gechebnet.liegroups.se2 import se2_uniform_sampling

In [ ]:
eps, xi = .1, 2.048 / (28 ** 2)

In [ ]:
se2_sampling = se2_uniform_sampling(28,28,6)

In [ ]:
se2_graph = SE2GEGraph(
    se2_sampling,
    K=32,
    sigmas=(1., eps, xi),
    path_to_graph="saved_graphs"
)

In [ ]:
visualize_graph(se2_graph)

In [ ]:
X, Y, Z = se2_graph.cartesian_pos()

In [ ]:
import matplotlib.cm as cm

num_neighborhoods = 6
num_layers = 6

fig = plt.figure(figsize=(20, 20))

for k in range(num_neighborhoods):
    neighbors_indices, neighbors_weights, _ = se2_graph.neighborhood(torch.randint(se2_graph.num_nodes, (1,)))
    weights = torch.zeros(se2_graph.num_nodes)
    weights[neighbors_indices] = neighbors_weights
    for l in range(num_layers):
        ax = fig.add_subplot(num_neighborhoods, num_layers, l * num_neighborhoods + k + 1)
        ax.scatter(X[l*784:(l+1)*784], Y[l*784:(l+1)*784], c=weights[l*784:(l+1)*784], cmap=cm.PuRd)
        ax.axis("off")

fig.tight_layout()
plt.show()


In [ ]:
visualize_graph_neighborhood(se2_graph, 55)

In [ ]:
visualize_graph_neighborhood(se2_graph, 455)

In [ ]:
se2_graph.node_x[455], se2_graph.node_y[455], se2_graph.node_theta[455]

In [ ]:
f"The SE(2) graph has {se2_graph.num_nodes} vertices and {se2_graph.num_edges} edges"

## SO(3) Group Equivariant Graph

The SO(3) group equivariant graph is the second group equivariant graph we have implemented. To create such a graph, different variables have to be speficied:
- `nsamples`: the discretization on the sphere, i.e. the number of points to uniformly sample on the sphere.
- `nalpha`: the discretization on the alpha axis, i.e. the number of points to uniformly sample in the alpha direction.
- `knn`: the maximum number of neighbors of each vertex. 
- `sigmas`: the anisotropy's parameters for the computation of the Riemannian distance between vertices.
- `weight_kernel`: the weight kernel, that is a function taking as input the squared Riemannian distance and returning a weight.
- `kappa`: the edge's compression rate, the rate of edges to drop during random compression.

In [ ]:
from gechebnet.liegroups.so3 import so3_uniform_sampling

In [ ]:
eps, xi = 1., 1.

In [ ]:
eps, xi = .1, 32.1 / (10 * 4 ** 3 + 2)

In [ ]:
path_to_sampling = "/home/hugo/Documents/thesis/GroupEquivariantChebNets/gechebnet/liegroups/sampling"
so3_sampling = so3_uniform_sampling(path_to_sampling, 3, 6)

In [ ]:
so3_graph = SO3GEGraph(
    so3_sampling,
    K=32,
    sigmas=(1., eps, xi),
    path_to_graph="saved_graphs"
)

In [ ]:
X, Y, Z = so3_graph.cartesian_pos()

In [ ]:
import matplotlib.cm as cm

num_neighborhoods = 6
num_layers = 6

fig = plt.figure(figsize=(20, 20))

for k in range(num_neighborhoods):
    neighbors_indices, neighbors_weights, _ = so3_graph.neighborhood(torch.randint(so3_graph.num_nodes, (1,)))
    weights = torch.zeros(so3_graph.num_nodes)
    weights[neighbors_indices] = neighbors_weights
    for l in range(num_layers):
        ax = fig.add_subplot(num_neighborhoods, num_layers, l * num_neighborhoods + k + 1, projection="3d")
        ax.scatter(X[l*642:(l+1)*642], Y[l*642:(l+1)*642], Z[l*642:(l+1)*642], c=weights[l*642:(l+1)*642], cmap=cm.PuRd)
        ax.axis("off")

fig.tight_layout()
plt.show()


In [ ]:
visualize_graph(so3_graph)

In [ ]:
visualize_graph_neighborhood(so3_graph, 100)

In [ ]:
f"The SO(3) graph has {so3_graph.num_nodes} vertices and {so3_graph.num_edges} edges"